In [1]:
import math
import numpy
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.stat import Statistics
from pyspark.sql import SQLContext
from numpy import array
from pyspark.sql import Row

In [2]:
data_rows = sc.textFile("/FileStore/tables/order_products__prior.csv")
data_rows.take(2)

Out[ 3 ]: [u'order_id,product_id,add_to_cart_order,reordered', u'2,33120,1,1']

In [3]:
#Remove the first line (contains headers)
dataLines = data_rows.filter(lambda x: "product_id" not in x)
dataLines.count()

Out[ 4 ]: 32434489

In [4]:
#Create 
def transformToVector(inputstr):
    attlist = inputstr.split(",")
    values = Vectors.dense([int(attlist[0]),int(attlist[1])])
    return values

In [5]:
#Change to a vector
dataVectors= dataLines.map(transformToVector)
dataVectors.take(2)

Out[ 6 ]: [DenseVector([2.0, 33120.0]), DenseVector([2.0, 28985.0])]

In [6]:
sqlcontext=SQLContext(sc)

In [7]:
def transformToLabeledPoint(instr):
    attList=instr.split(",")
    lp = (int(attList[0]),int(attList[1]))
    return lp

In [8]:
dataLp=dataLines.map(transformToLabeledPoint)
dataLp.take(5)

Out[ 9 ]: [(2, 33120), (2, 28985), (2, 9327), (2, 45918), (2, 30035)]

In [9]:
datatDF=sqlContext.createDataFrame(dataLp,["order_id","product_id"])

In [10]:
datatDF.select("order_id","product_id").show(2)

+--------+----------+
order_id|product_id|
+--------+----------+
 2| 33120|
 2| 28985|
+--------+----------+
only showing top 2 rows

In [11]:
import pandas as pd
import numpy as np
import sys
from itertools import combinations, groupby
from collections import Counter
from IPython.display import display

In [12]:
datatDF.describe()

Out[ 36 ]: DataFrame[summary: string, order_id: string, product_id: string]

In [13]:
data = datatDF.limit(7000000).toPandas()

In [14]:
def size(obj):
    return "{0:.2f} MB".format(sys.getsizeof(obj) / (1000 * 1000))

In [15]:
print('orders -- dimensions: {0};   size: {1}'.format(data.shape, size(data)))

orders -- dimensions: (7000000, 2); size: 112.00 MB

In [16]:
# Convert from DataFrame to a Series, with order_id as index and item_id as value
orders = data.set_index('order_id')['product_id'].rename('item_id')

In [17]:
display(orders.head(20))
type(orders)

order_id
2 33120
2 28985
2 9327
2 45918
2 30035
2 17794
2 40141
2 1819
2 43668
3 33754
3 24838
3 17704
3 21903
3 17668
3 46667
3 17461
3 32665
4 46842
4 26434
4 39758
Name: item_id, dtype: int64
 Out[ 22 ]: pandas.core.series.Series

In [18]:
print('dimensions: {0};   size: {1};   unique_orders: {2};   unique_items: {3}'
      .format(orders.shape, size(orders), len(orders.index.unique()), len(orders.value_counts())))

dimensions: (7000000,); size: 112.00 MB; unique_orders: 694184; unique_items: 47470

In [19]:
# Returns frequency counts for items and item pairs
def freq(iterable):
    if type(iterable) == pd.core.series.Series:
        return iterable.value_counts().rename("freq")
    else: 
        return pd.Series(Counter(iterable)).rename("freq")


In [20]:
# Returns number of unique orders
def order_count(order_item):
    return len(set(order_item.index))

In [21]:
# Returns generator that yields item pairs, one at a time
def get_item_pairs(order_item):
    order_item = order_item.reset_index().as_matrix()
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]
              
        for item_pair in combinations(item_list, 2):
            yield item_pair

In [22]:
# Returns frequency and support associated with item
def merge_item_stats(item_pairs, item_stats):
    return (item_pairs
                .merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}), left_on='item_A', right_index=True)
                .merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}), left_on='item_B', right_index=True))

In [23]:
# Returns name associated with item
def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB', 
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules
                .merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
                .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))
    return rules[columns]      

In [24]:
def association_rules(order_item, min_support):

    print("Starting order_item: {:22d}".format(len(order_item)))


    # Calculate item frequency and support
    item_stats = freq(order_item).to_frame("freq")
    item_stats['support'] = item_stats['freq'] / order_count(order_item) * 100


    # Filter from order_item items below min support 
    qualifying_items = item_stats[item_stats['support'] >= min_support].index
    order_item = order_item[order_item.isin(qualifying_items)]

    print("Items with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Filter from order_item orders with less than 2 items
    order_size = freq(order_item.index)
    qualifying_orders = order_size[order_size >= 2].index
    order_item = order_item[order_item.index.isin(qualifying_orders)]

    print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_orders)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Recalculate item frequency and support
    item_stats = freq(order_item).to_frame("freq")
    item_stats['support'] = item_stats['freq'] / order_count(order_item) * 100


    # Get item pairs generator
    item_pair_gen = get_item_pairs(order_item)


    # Calculate item pair frequency and support
    item_pairs = freq(item_pair_gen).to_frame("freqAB")
    item_pairs['supportAB'] = item_pairs['freqAB'] / len(qualifying_orders) * 100

    print("Item pairs: {:31d}".format(len(item_pairs)))


    # Filter from item_pairs those below min support
    item_pairs = item_pairs[item_pairs['supportAB'] >= min_support]

    print("Item pairs with support >= {}: {:10d}\n".format(min_support, len(item_pairs)))


    # Create table of association rules and compute relevant metrics
    item_pairs = item_pairs.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    item_pairs = merge_item_stats(item_pairs, item_stats)
    
    item_pairs['confidenceAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confidenceBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift']           = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])
    
    
    # Return association rules sorted by lift in descending order
    return item_pairs.sort_values('lift', ascending=False)

In [25]:
%%time
rules = association_rules(orders, 0.04)  

Starting order_item: 7000000
Items with support >= 0.04: 4123
Remaining order_item: 5494313
Remaining orders with 2+ items: 627718
Remaining order_item: 5443208
Item pairs: 6422989
Item pairs with support >= 0.04: 6530

CPU times: user 1min 32s, sys: 6.64 s, total: 1min 38s
Wall time: 1min 36s

In [26]:
rules.head(10)

Out[ 43 ]: 
 item_A item_B freqAB supportAB freqA supportA freqB supportB \
4815 38312 15984 365 0.058147 1253 0.199612 1036 0.165042 
6363 48220 15984 289 0.046040 1049 0.167113 1036 0.165042 
4816 38312 48220 308 0.049067 1253 0.199612 1049 0.167113 
6364 48220 38312 256 0.040783 1049 0.167113 1253 0.199612 
206 4962 38544 360 0.057351 1317 0.209808 1412 0.224942 
5402 44156 23296 357 0.056873 1649 0.262698 1354 0.215702 
4828 38544 4962 295 0.046996 1412 0.224942 1317 0.209808 
5793 47029 48726 265 0.042216 1459 0.232429 1286 0.204869 
4487 33548 23296 311 0.049545 1685 0.268433 1354 0.215702 
3434 26131 8490 298 0.047474 1612 0.256803 1408 0.224305 

 confidenceAtoB confidenceBtoA lift 
4815 0.291301 0.352317 1.765008 
6363 0.275500 0.278958 1.669272 
4816 0.245810 0.293613 1.470919 
6364 0.244042 0.204310 1.222582 
206 0.273349 0.254958 1.215197 
5402 0.216495 0.263663 1.003676 
4828 0.208924 0.223994 0.995786 
5793 0.181631 0.206065 0.886572 
4487 0.184570 0.229690 0.855670 
3434 0.184864 0.211648 0.824163